In [2]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd
dataset = pd.read_excel(r'new.xlsx'  )
smiles = dataset['SMILES']
mols = [Chem.MolFromSmiles(m) for m in smiles]
def getDefinedMolDescriptors(mol, list, missingVal=None):
    ''' calculate the list of descriptors for a molecule
        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in list:
        try:
            val = fn(mol)
        except:
            import traceback
            traceback.print_exc()
            val = missingVal
        res[nm] = val
    return res
# new decriptors
desc_list = ['NumHAcceptors','NumHDonors']
selected_list = [(name, func) for name, func in Descriptors._descList if name in desc_list]
definedDescrs = [getDefinedMolDescriptors(m, selected_list) for m in mols]
df2 = pd.DataFrame(definedDescrs)
# one-hot encoding of anchor_end
unique_anchors = ['anchor_end_' + anchor for anchor in dataset['anchor_end'].str.split(';').explode().unique()] # ['anchor_end_PA','anchor_end_SA','anchor_end_CA','anchor_end_BA','anchor_end_O','anchor_end_SiO','anchor_end_N','anchor_end_S']
one_hot_df = pd.DataFrame(index=dataset.index, columns=unique_anchors, dtype=int)
# Fill the one-hot encoded DataFrame
for index, row in dataset.iterrows():
    for anchor in row['anchor_end'].split(';'):
        one_hot_df.loc[index, 'anchor_end_' + anchor] = 1
one_hot_df.fillna(0, inplace=True)
data = pd.concat([df2, one_hot_df, pd.get_dummies(dataset.iloc[:,6:],dtype=int)],axis=1)
data.drop(['mol_vol','NPSA_multiwfn'],axis=1,inplace=True)
data = data[list(data.columns[data.columns != 'PCE'])+['PCE']]
data.to_csv('dataset.csv',index=False)
